# Deep-Sea Exploration Analysis Demo

This notebook demonstrates the core functionality of the exploration planner using the Python API.

In [ ]:
import sys
import json
import pandas as pd
sys.path.append('../src')

from scoring import SitePrioritizationEngine
from matching import RobotSiteMatcher
from optimizer import MissionOptimizer

## Load Data

In [ ]:
# Load sites and robots
with open('../data/sites.json', 'r') as f:
    sites = json.load(f)

with open('../data/robots.json', 'r') as f:
    robots = json.load(f)

print(f"Loaded {len(sites)} sites and {len(robots)} robots")

## Site Prioritization

In [ ]:
# Initialize scoring engine
scorer = SitePrioritizationEngine()

# Score all sites
scored_sites = scorer.score_sites(sites)

# Display top 5 sites
scored_sites[['name', 'composite_score', 'mineral_concentration', 'estimated_value_millions']].head()

## Robot-Site Matching

In [ ]:
# Initialize matcher
matcher = RobotSiteMatcher(robots, sites)

# Get compatibility matrix
compatibility = matcher.get_compatibility_matrix()

compatibility.head()

## Mission Optimization

In [ ]:
# Initialize optimizer
optimizer = MissionOptimizer(robots, scored_sites.to_dict('records'), compatibility)

# Run optimization
result = optimizer.greedy_mission_scheduler(
    time_window_days=180,
    budget_usd=10000000,
    vessel_speed_knots=12,
    prioritize_by='score'
)

print(f"Scheduled {result['statistics']['total_missions']} missions")
print(f"Coverage: {result['statistics']['coverage_percent']:.1f}%")
print(f"Total Cost: ${result['statistics']['total_cost_usd']:,.0f}")

In [ ]:
# Convert missions to DataFrame for analysis
missions_df = pd.DataFrame(result['missions'])
missions_df[['mission_id', 'robot_name', 'site_name', 'total_hours', 'cost_usd']]